# Simple template for renaming DFs

In [1]:
# read in all the dfs from signal_events_old.h5 and rename one of the groups
import sys,os,os.path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
from tqdm import tqdm

import csv
import traceback

sys.path.append("../")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

from invisible_cities.core.core_functions   import shift_to_bin_centers
from invisible_cities.io.dst_io           import load_dst, load_dsts, df_writer

import core.loading_functions as load_func
import quantification.quantify_functions as quant_func


In [3]:
full_path = f'../data/CLEAN/signal_events_old.h5'
MCpart             = pd.read_hdf(full_path, 'MC/particles')
RECOevt            = pd.read_hdf(full_path, 'RECO/Events')
MChits             = pd.read_hdf(full_path, 'MC/hits')
TrackingMC_Tracks  = pd.read_hdf(full_path, 'Tracking/Tracks')

display(MCpart)
display(RECOevt)
display(MChits)
display(TrackingMC_Tracks)

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,12300006,1,Tl208,True,0,398.747284,401.567871,563.659546,0.000000,398.747284,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
1,12300006,4,e-,False,1,398.747284,401.567871,563.659546,0.000762,398.747589,...,-0.017302,-0.296512,-0.155331,-0.000000,0.000000,0.000000,0.100120,2.790027e-02,RadioactiveDecay,NoProcess
2,12300006,3,anti_nu_e,False,1,398.747284,401.567871,563.659546,0.000762,-9720.689453,...,-0.919971,-0.797942,-1.187634,-0.919971,-0.797942,-1.187634,1.701039,1.871097e+04,RadioactiveDecay,Transportation
3,12300006,2,Pb208[3197.711],False,1,398.747284,401.567871,563.659546,0.000762,398.747284,...,0.937273,1.094454,1.342965,0.000000,0.000000,0.000000,0.000010,0.000000e+00,RadioactiveDecay,RadioactiveDecay
4,12300006,6,gamma,False,2,398.747284,401.567871,563.659546,0.000762,398.441864,...,-0.275587,-0.117285,0.500404,-0.000000,-0.000000,0.000000,0.583188,6.463141e-01,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49929,12204085,58991,e-,False,58504,246.444626,434.608490,1265.968140,5.705839,246.519730,...,-0.134671,0.556736,0.245645,-0.000000,0.000000,0.000000,0.294949,4.999868e-01,compt,NoProcess
49930,12204085,69,e-,False,66,208.218964,-34.900761,997.138367,2.151887,209.444992,...,-0.233796,-0.094158,-0.059741,-0.000000,0.000000,0.000000,0.061902,6.112734e+00,conv,NoProcess
49931,12204085,68,e-,False,66,384.466278,339.716492,633.618896,0.314111,384.466278,...,0.004220,0.003004,-0.002740,0.000000,0.000000,-0.000000,0.000034,6.015512e-08,compt,NoProcess
49932,12204085,67,e-,False,66,384.466278,339.716492,633.618896,0.314111,384.484070,...,0.379871,-0.092510,0.356779,0.000000,-0.000000,-0.000000,0.224713,1.074977e-01,compt,NoProcess


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-155.375,0.0,0.0,40.263651,7.188406,574.714427,-1.0,0.001402,-1,-1.0
1,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-139.825,0.0,0.0,40.263651,5.217391,417.131443,-1.0,0.001018,-1,-1.0
2,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-155.375,0.0,0.0,40.263651,15.072464,1205.046364,-1.0,0.002943,-1,-1.0
3,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-139.825,0.0,0.0,40.263651,9.449275,755.471336,-1.0,0.001847,-1,-1.0
4,12300006,2.460001e+13,0,6.515505,-148.908484,1,10.975,-155.375,0.0,0.0,40.263651,8.811594,704.488618,-1.0,0.001721,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420832,12204085,2.440817e+13,1,200.640000,-7.964710,1,288.875,-16.925,0.0,0.0,1028.999786,6.608695,398.112332,-1.0,0.001020,-1,-1.0
420833,12204085,2.440817e+13,1,200.640000,-7.964710,1,304.425,-16.925,0.0,0.0,1028.999786,5.565217,335.252499,-1.0,0.000870,-1,-1.0
420834,12204085,2.440817e+13,1,200.640000,-7.964710,1,257.775,-16.925,0.0,0.0,1032.469781,5.855072,1413.585331,-1.0,0.003604,-1,-1.0
420835,12204085,2.440817e+13,1,200.640000,-7.964710,1,257.775,-1.375,0.0,0.0,1032.469781,5.623188,1357.601767,-1.0,0.003466,-1,-1.0


,event,particle_id,hit_id,X,Y,Z,time,E,label,npeak
0,12300006,32,0,36.513824,-125.199722,105.147636,2.834566,0.000001,ACTIVE,1
1,12300006,31,0,36.547150,-125.759644,104.332382,2.629001,0.002518,ACTIVE,1
2,12300006,31,1,36.605511,-126.437218,103.599846,2.633066,0.002213,ACTIVE,1
3,12300006,31,2,36.760361,-127.064156,102.854980,2.637081,0.002263,ACTIVE,1
4,12300006,31,3,37.286903,-127.554703,102.174568,2.641119,0.000403,ACTIVE,1
...,...,...,...,...,...,...,...,...,...,...
504869,12204085,69,52,209.448700,-34.775208,997.616333,2.203594,0.002556,ACTIVE,1
504870,12204085,69,53,209.446777,-34.771420,997.600708,2.203996,0.002089,ACTIVE,1
504871,12204085,69,54,209.446533,-34.771984,997.601624,2.204033,0.000024,ACTIVE,1
504872,12204085,69,55,209.445343,-34.767570,997.604004,2.204250,0.002396,ACTIVE,1


,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,1212003,0,1.551080,497.286267,511,967,2,155.699524,227.459152,561.196777,...,576.539307,194.930542,238.221573,672.626404,0.292241,0.236851,0.000000,0.996020,0.997924,0.997965
1,1212003,1,0.041426,0.000000,1,82,2,211.962662,185.492798,696.540649,...,696.766026,212.022651,185.602417,696.766026,0.041426,0.041426,0.041426,0.996020,0.997924,0.997965
2,1212019,0,1.062401,374.265850,352,649,1,83.538750,139.741592,574.431152,...,628.451660,216.694138,242.315384,574.695740,0.233463,0.184444,0.000000,0.999854,0.996397,0.991111
3,1212037,0,1.592507,626.026621,597,972,5,194.850876,74.949455,230.235168,...,358.229431,341.690918,205.188858,240.594391,0.233390,0.224443,0.000000,0.999774,0.998009,0.999099
4,1212037,1,0.116812,7.554383,14,106,5,387.340179,294.523743,634.873535,...,635.005979,390.522777,294.691948,641.010558,0.116812,0.116812,0.116812,0.999774,0.998009,0.999099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,12336087,0,1.592507,548.400899,536,779,3,198.715042,115.408508,475.540802,...,538.499878,207.464091,171.687855,488.056352,0.194041,0.193727,0.000000,0.995961,0.998120,0.998279
1356,12336087,1,0.140402,4.865310,14,118,3,276.328186,208.832825,833.028992,...,835.246704,279.190606,210.913630,833.314586,0.140402,0.140402,0.140402,0.995961,0.998120,0.998279
1357,12336087,2,0.090864,3.723881,10,62,3,258.667389,196.071686,805.258484,...,805.933119,260.316284,196.071686,806.529724,0.090864,0.090864,0.090864,0.995961,0.998120,0.998279
1358,12336095,0,1.592507,481.136755,536,927,2,223.911438,194.934479,461.894897,...,550.468201,254.417557,257.600708,479.306793,0.324534,0.195551,0.000000,0.997759,0.997815,0.998575


In [4]:
output_file = '../data/CLEAN/signal_events.h5'
with tb.open_file(output_file, 'w') as h5out:
    df_writer(h5out, MCpart, 'MC', 'particles')
    df_writer(h5out, RECOevt, 'RECO', 'Events')
    df_writer(h5out, MChits, 'MC', 'hits')
    df_writer(h5out, TrackingMC_Tracks, 'Tracking', 'MC_Tracks')
    